In [1]:
# /Reproducing/preprocessing/GridSat_B1_preprocessing
import glob
import os
import netCDF4 as nc
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

# Setting

In [2]:
root_path = '/mnt/sdc/GridSat-B1/'
new_path = '/home/ices/work/typhoon_intensity/data/GridSat_B1_npy/'
new2_path = '/home/ices/work/typhoon_intensity/data/GridSat_B1_new_npy/'

In [3]:
train_range = (2010, 2017)
test_range = (2018, 2020)

# Sample

In [4]:
# sample = nc.Dataset('/mnt/sdc/GridSat-B1/2007/GRIDSAT-B1.2007.01.01.00.v02r01.nc')

In [5]:
# plt.figure(figsize=(16,16))
# plt.imshow(sample.variables['irwin_cdr'][0].data, cmap = plt.cm.gray)
# plt.axis('off')
# plt.show()

In [6]:
all_info = pd.read_csv('/home/ices/work/typhoon_intensity/data/ibtracs.ALL.list.v04r00.csv', skiprows=[1], na_values=' ')

/home/ices/anaconda3/envs/typhoon_intensity/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3098: DtypeWarning: Columns (21,61,66,128,143,148) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
all_info

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,...,BOM_GUST_PER,REUNION_GUST,REUNION_GUST_PER,USA_SEAHGT,USA_SEARAD_NE,USA_SEARAD_SE,USA_SEARAD_SW,USA_SEARAD_NW,STORM_SPEED,STORM_DIR
0,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 03:00:00,NR,10.9000,80.3000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,266.0
1,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 06:00:00,NR,10.8709,79.8265,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,267.0
2,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 09:00:00,NR,10.8431,79.3524,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,267.0
3,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 12:00:00,NR,10.8188,78.8772,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,267.0
4,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 15:00:00,NR,10.8000,78.4000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,268.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698343,2021198N13257,2021,40,EP,MM,GUILLERMO,2021-07-17 12:00:00,TS,16.4200,-108.3200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,297.0
698344,2021198N13257,2021,40,EP,MM,GUILLERMO,2021-07-17 15:00:00,TS,16.6818,-108.9270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,293.0
698345,2021198N13257,2021,40,EP,MM,GUILLERMO,2021-07-17 18:00:00,TS,16.9000,-109.5000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,291.0
698346,2021198N13257,2021,40,EP,MM,GUILLERMO,2021-07-17 21:00:00,TS,17.1030,-110.0530,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,291.0


In [8]:
all_info['ISO_TIME']=all_info['ISO_TIME'].apply(lambda x:datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
all_info['YEAR']=all_info['ISO_TIME'].apply(lambda x:x.year)
all_info['TIME']=all_info['ISO_TIME'].apply(lambda x:datetime.datetime.strftime(x,'%m%d%H'))
all_info['time_dot']=all_info['ISO_TIME'].apply(lambda x:datetime.datetime.strftime(x,'%Y.%m.%d.%H'))

In [9]:
all_info

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,...,USA_SEAHGT,USA_SEARAD_NE,USA_SEARAD_SE,USA_SEARAD_SW,USA_SEARAD_NW,STORM_SPEED,STORM_DIR,YEAR,TIME,time_dot
0,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 03:00:00,NR,10.9000,80.3000,...,NaN,NaN,NaN,NaN,NaN,9.0,266.0,1842,102503,1842.10.25.03
1,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 06:00:00,NR,10.8709,79.8265,...,NaN,NaN,NaN,NaN,NaN,9.0,267.0,1842,102506,1842.10.25.06
2,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 09:00:00,NR,10.8431,79.3524,...,NaN,NaN,NaN,NaN,NaN,9.0,267.0,1842,102509,1842.10.25.09
3,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 12:00:00,NR,10.8188,78.8772,...,NaN,NaN,NaN,NaN,NaN,9.0,267.0,1842,102512,1842.10.25.12
4,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 15:00:00,NR,10.8000,78.4000,...,NaN,NaN,NaN,NaN,NaN,9.0,268.0,1842,102515,1842.10.25.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698343,2021198N13257,2021,40,EP,MM,GUILLERMO,2021-07-17 12:00:00,TS,16.4200,-108.3200,...,NaN,NaN,NaN,NaN,NaN,14.0,297.0,2021,071712,2021.07.17.12
698344,2021198N13257,2021,40,EP,MM,GUILLERMO,2021-07-17 15:00:00,TS,16.6818,-108.9270,...,NaN,NaN,NaN,NaN,NaN,12.0,293.0,2021,071715,2021.07.17.15
698345,2021198N13257,2021,40,EP,MM,GUILLERMO,2021-07-17 18:00:00,TS,16.9000,-109.5000,...,NaN,NaN,NaN,NaN,NaN,12.0,291.0,2021,071718,2021.07.17.18
698346,2021198N13257,2021,40,EP,MM,GUILLERMO,2021-07-17 21:00:00,TS,17.1030,-110.0530,...,NaN,NaN,NaN,NaN,NaN,11.0,291.0,2021,071721,2021.07.17.21


In [10]:
match_info = all_info[(all_info['YEAR'] >= train_range[0])&(all_info['YEAR'] <= test_range[1])].copy()
match_info = match_info.reset_index(drop=True)
match_info

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,...,USA_SEAHGT,USA_SEARAD_NE,USA_SEARAD_SE,USA_SEARAD_SW,USA_SEARAD_NW,STORM_SPEED,STORM_DIR,YEAR,TIME,time_dot
0,2010002S09096,2010,1,SI,WA,EDZANI,2010-01-01 12:00:00,NR,-9.20000,95.7000,...,NaN,NaN,NaN,NaN,NaN,7.0,210.0,2010,010112,2010.01.01.12
1,2010002S09096,2010,1,SI,WA,EDZANI,2010-01-01 15:00:00,NR,-9.51501,95.5176,...,NaN,NaN,NaN,NaN,NaN,7.0,213.0,2010,010115,2010.01.01.15
2,2010002S09096,2010,1,SI,WA,EDZANI,2010-01-01 18:00:00,TS,-9.80000,95.3000,...,NaN,NaN,NaN,NaN,NaN,7.0,223.0,2010,010118,2010.01.01.18
3,2010002S09096,2010,1,SI,WA,EDZANI,2010-01-01 21:00:00,TS,-10.03000,95.0225,...,NaN,NaN,NaN,NaN,NaN,7.0,236.0,2010,010121,2010.01.01.21
4,2010002S09096,2010,1,SI,WA,EDZANI,2010-01-02 00:00:00,TS,-10.20000,94.7000,...,NaN,NaN,NaN,NaN,NaN,7.0,245.0,2010,010200,2010.01.02.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67416,2020360S16057,2021,53,SI,MM,CHALANE,2020-12-29 12:00:00,NR,-19.60000,38.5000,...,NaN,NaN,NaN,NaN,NaN,10.0,264.0,2020,122912,2020.12.29.12
67417,2020360S16057,2021,53,SI,MM,CHALANE,2020-12-29 15:00:00,NR,-19.68650,38.1155,...,NaN,NaN,NaN,NaN,NaN,8.0,262.0,2020,122915,2020.12.29.15
67418,2020360S16057,2021,53,SI,MM,CHALANE,2020-12-29 18:00:00,NR,-19.70000,37.7000,...,NaN,NaN,NaN,NaN,NaN,11.0,279.0,2020,122918,2020.12.29.18
67419,2020360S16057,2021,53,SI,MM,CHALANE,2020-12-29 21:00:00,NR,-19.51400,36.9938,...,NaN,NaN,NaN,NaN,NaN,16.0,288.0,2020,122921,2020.12.29.21


In [11]:
match_info['lat_mid'] = np.round(match_info['LAT'] / 0.07 + 1000).astype('int64')
match_info['long_mid'] = np.round((match_info['LON'] + 180) / 0.07 ).astype('int64')
match_info

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,...,USA_SEARAD_SE,USA_SEARAD_SW,USA_SEARAD_NW,STORM_SPEED,STORM_DIR,YEAR,TIME,time_dot,lat_mid,long_mid
0,2010002S09096,2010,1,SI,WA,EDZANI,2010-01-01 12:00:00,NR,-9.20000,95.7000,...,NaN,NaN,NaN,7.0,210.0,2010,010112,2010.01.01.12,869,3939
1,2010002S09096,2010,1,SI,WA,EDZANI,2010-01-01 15:00:00,NR,-9.51501,95.5176,...,NaN,NaN,NaN,7.0,213.0,2010,010115,2010.01.01.15,864,3936
2,2010002S09096,2010,1,SI,WA,EDZANI,2010-01-01 18:00:00,TS,-9.80000,95.3000,...,NaN,NaN,NaN,7.0,223.0,2010,010118,2010.01.01.18,860,3933
3,2010002S09096,2010,1,SI,WA,EDZANI,2010-01-01 21:00:00,TS,-10.03000,95.0225,...,NaN,NaN,NaN,7.0,236.0,2010,010121,2010.01.01.21,857,3929
4,2010002S09096,2010,1,SI,WA,EDZANI,2010-01-02 00:00:00,TS,-10.20000,94.7000,...,NaN,NaN,NaN,7.0,245.0,2010,010200,2010.01.02.00,854,3924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67416,2020360S16057,2021,53,SI,MM,CHALANE,2020-12-29 12:00:00,NR,-19.60000,38.5000,...,NaN,NaN,NaN,10.0,264.0,2020,122912,2020.12.29.12,720,3121
67417,2020360S16057,2021,53,SI,MM,CHALANE,2020-12-29 15:00:00,NR,-19.68650,38.1155,...,NaN,NaN,NaN,8.0,262.0,2020,122915,2020.12.29.15,719,3116
67418,2020360S16057,2021,53,SI,MM,CHALANE,2020-12-29 18:00:00,NR,-19.70000,37.7000,...,NaN,NaN,NaN,11.0,279.0,2020,122918,2020.12.29.18,719,3110
67419,2020360S16057,2021,53,SI,MM,CHALANE,2020-12-29 21:00:00,NR,-19.51400,36.9938,...,NaN,NaN,NaN,16.0,288.0,2020,122921,2020.12.29.21,721,3100


In [12]:
match_info['lat_north'] = match_info['lat_mid'] + 150
match_info['lat_south'] = match_info['lat_mid'] - 150
match_info['long_west'] = match_info['long_mid'] - 150
match_info['long_east'] = match_info['long_mid'] + 150
match_info

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,...,STORM_DIR,YEAR,TIME,time_dot,lat_mid,long_mid,lat_north,lat_south,long_west,long_east
0,2010002S09096,2010,1,SI,WA,EDZANI,2010-01-01 12:00:00,NR,-9.20000,95.7000,...,210.0,2010,010112,2010.01.01.12,869,3939,1019,719,3789,4089
1,2010002S09096,2010,1,SI,WA,EDZANI,2010-01-01 15:00:00,NR,-9.51501,95.5176,...,213.0,2010,010115,2010.01.01.15,864,3936,1014,714,3786,4086
2,2010002S09096,2010,1,SI,WA,EDZANI,2010-01-01 18:00:00,TS,-9.80000,95.3000,...,223.0,2010,010118,2010.01.01.18,860,3933,1010,710,3783,4083
3,2010002S09096,2010,1,SI,WA,EDZANI,2010-01-01 21:00:00,TS,-10.03000,95.0225,...,236.0,2010,010121,2010.01.01.21,857,3929,1007,707,3779,4079
4,2010002S09096,2010,1,SI,WA,EDZANI,2010-01-02 00:00:00,TS,-10.20000,94.7000,...,245.0,2010,010200,2010.01.02.00,854,3924,1004,704,3774,4074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67416,2020360S16057,2021,53,SI,MM,CHALANE,2020-12-29 12:00:00,NR,-19.60000,38.5000,...,264.0,2020,122912,2020.12.29.12,720,3121,870,570,2971,3271
67417,2020360S16057,2021,53,SI,MM,CHALANE,2020-12-29 15:00:00,NR,-19.68650,38.1155,...,262.0,2020,122915,2020.12.29.15,719,3116,869,569,2966,3266
67418,2020360S16057,2021,53,SI,MM,CHALANE,2020-12-29 18:00:00,NR,-19.70000,37.7000,...,279.0,2020,122918,2020.12.29.18,719,3110,869,569,2960,3260
67419,2020360S16057,2021,53,SI,MM,CHALANE,2020-12-29 21:00:00,NR,-19.51400,36.9938,...,288.0,2020,122921,2020.12.29.21,721,3100,871,571,2950,3250


In [13]:
(match_info['lat_north'] > 0).all

<bound method NDFrame._add_numeric_operations.<locals>.all of 0        True
1        True
2        True
3        True
4        True
         ... 
67416    True
67417    True
67418    True
67419    True
67420    True
Name: lat_north, Length: 67421, dtype: bool>

In [14]:
match_info = match_info[~match_info['USA_WIND'].isnull()]
# match_info = match_info[match_info['SEASON']!=2016]
match_info = match_info.reset_index(drop=True)
match_info.shape

(58371, 172)

In [15]:
match_info.describe()

,SEASON,NUMBER,LAT,LON,WMO_WIND,WMO_PRES,DIST2LAND,LANDFALL,USA_LAT,USA_LON,...,USA_SEARAD_NW,STORM_SPEED,STORM_DIR,YEAR,lat_mid,long_mid,lat_north,lat_south,long_west,long_east
count,58371.000000,58371.000000,58371.000000,58371.000000,26124.000000,27850.000000,58371.000000,57680.000000,58371.000000,58371.000000,...,10146.000000,58371.000000,58371.000000,58371.000000,58371.000000,58371.000000,58371.000000,58371.000000,58371.000000,58371.000000
mean,2015.182111,50.336982,11.059458,29.320933,50.140637,988.740395,806.158897,791.692892,11.063581,19.965195,...,158.878770,9.842747,221.049939,2015.132018,1157.992856,2990.295883,1307.992856,1007.992856,2840.295883,3140.295883
std,3.071146,29.610440,18.223105,112.054469,24.317542,20.804852,677.565383,675.432080,18.228464,112.830491,...,111.071429,6.027928,104.188858,3.066139,260.328525,1600.775998,260.328525,260.328525,1600.775998,1600.775998
min,2010.000000,1.000000,-40.750000,-179.750000,3.000000,872.000000,0.000000,0.000000,-41.000000,-180.000000,...,10.000000,0.000000,0.000000,2010.000000,418.000000,4.000000,568.000000,268.000000,-146.000000,154.000000
25%,2013.000000,25.000000,4.249080,-85.082100,30.000000,980.000000,239.000000,225.000000,4.300000,-94.573750,...,80.000000,6.000000,141.000000,2012.000000,1061.000000,1356.000000,1211.000000,911.000000,1206.000000,1506.000000
50%,2015.000000,50.000000,15.692400,68.540000,45.000000,996.000000,623.000000,608.000000,15.700000,61.000000,...,120.000000,9.000000,268.000000,2015.000000,1224.000000,3551.000000,1374.000000,1074.000000,3401.000000,3701.000000
75%,2018.000000,74.000000,22.259000,129.270000,65.000000,1004.000000,1287.000000,1269.000000,22.281300,126.400000,...,210.000000,13.000000,297.000000,2018.000000,1318.000000,4418.000000,1468.000000,1168.000000,4268.000000,4568.000000
max,2021.000000,116.000000,69.000000,244.900000,185.000000,1021.000000,3660.000000,3660.000000,69.000000,180.000000,...,740.000000,81.000000,360.000000,2020.000000,1986.000000,6070.000000,2136.000000,1836.000000,5920.000000,6220.000000


In [16]:
for c in match_info.columns:
    print(c)

SID
SEASON
NUMBER
BASIN
SUBBASIN
NAME
ISO_TIME
NATURE
LAT
LON
WMO_WIND
WMO_PRES
WMO_AGENCY
TRACK_TYPE
DIST2LAND
LANDFALL
IFLAG
USA_AGENCY
USA_ATCF_ID
USA_LAT
USA_LON
USA_RECORD
USA_STATUS
USA_WIND
USA_PRES
USA_SSHS
USA_R34_NE
USA_R34_SE
USA_R34_SW
USA_R34_NW
USA_R50_NE
USA_R50_SE
USA_R50_SW
USA_R50_NW
USA_R64_NE
USA_R64_SE
USA_R64_SW
USA_R64_NW
USA_POCI
USA_ROCI
USA_RMW
USA_EYE
TOKYO_LAT
TOKYO_LON
TOKYO_GRADE
TOKYO_WIND
TOKYO_PRES
TOKYO_R50_DIR
TOKYO_R50_LONG
TOKYO_R50_SHORT
TOKYO_R30_DIR
TOKYO_R30_LONG
TOKYO_R30_SHORT
TOKYO_LAND
CMA_LAT
CMA_LON
CMA_CAT
CMA_WIND
CMA_PRES
HKO_LAT
HKO_LON
HKO_CAT
HKO_WIND
HKO_PRES
NEWDELHI_LAT
NEWDELHI_LON
NEWDELHI_GRADE
NEWDELHI_WIND
NEWDELHI_PRES
NEWDELHI_CI
NEWDELHI_DP
NEWDELHI_POCI
REUNION_LAT
REUNION_LON
REUNION_TYPE
REUNION_WIND
REUNION_PRES
REUNION_TNUM
REUNION_CI
REUNION_RMW
REUNION_R34_NE
REUNION_R34_SE
REUNION_R34_SW
REUNION_R34_NW
REUNION_R50_NE
REUNION_R50_SE
REUNION_R50_SW
REUNION_R50_NW
REUNION_R64_NE
REUNION_R64_SE
REUNION_R64_SW
REUNION_R

# nc to npy

In [17]:
import logging
# https://www.cnblogs.com/qianyuliang/p/7234217.html
logger = logging.getLogger(__name__)
logger.setLevel(level = logging.INFO)
handler = logging.FileHandler("GridSat_B1_processor.log")
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
 
console = logging.StreamHandler()
console.setLevel(logging.INFO)
 
logger.addHandler(handler)
logger.addHandler(console)

In [18]:
# for idx in match_info.index:    
#     tc_data = match_info.loc[idx]
#     nc_path = f"{root_path}{tc_data['YEAR']}/GRIDSAT-B1.{tc_data['time_dot']}.v02r01.nc"  
#     logger.info(nc_path)
#     if os.path.exists(nc_path):
#         # 读nc文件的irwin_2
#         data = nc.Dataset(nc_path,format = 'NETCDF4')
#         array_all = data.variables['irwin_cdr'][:].data
        
#         # 东侧的索引值更大
#         west = tc_data['long_west']
#         east = tc_data['long_east'] + 1
#         if west < 0:
#             # 在经度上重复一遍
#             array_all = np.tile(array_all,2)
#             # 转化为正索引
#             west = west + 5143
#             east = east + 5143
#         elif east > 5142:
#             # 在经度上重复一遍
#             array_all = np.tile(array_all,2)
#         if tc_data['lat_north'] > 1999:
#             array_all = np.concatenate((array_all,np.zeros(array_all.shape)-31999), axis = 1)
#         # array_all.shape (1, 2000, 5143)
#         # 北侧的索引值更大
#         array_tc_original = np.ascontiguousarray(array_all[:,tc_data['lat_north']:tc_data['lat_south']-1:-1 ,west:east])
#         array_tc = array_tc_original * 0.01 + 200
        
#         path_sid = f"{new_path}{tc_data['SID']}"
#         array_file = f"{path_sid}/{tc_data['SID']}.{tc_data['YEAR']}{tc_data['TIME']}00.npy"
# #         print(array_tc.shape)
# #         print(path_sid)
# #         print(array_file)
#         os.makedirs(path_sid, exist_ok=True)
#         if os.path.exists(array_file) is False: 
#             np.save(array_file, array_tc)
#             logger.info(f"Numpy array saved,{array_file}")
#         else:
#             logger.info(f"Numpy array exist,{array_file}")
#     else:
#         logger.info(f"File Error!,{nc_path}")

In [19]:
# irwvp
# vschn
def processing_nc(varible_name='irwin_cdr'):
    data = nc.Dataset(nc_path,format = 'NETCDF4')
    array_all = data.variables[varible_name][:].data

    # 东侧的索引值更大
    west = tc_data['long_west']
    east = tc_data['long_east'] + 1
    if west < 0:
        # 在经度上重复一遍
        array_all = np.tile(array_all,2)
        # 转化为正索引
        west = west + 5143
        east = east + 5143
    elif east > 5142:
        # 在经度上重复一遍
        array_all = np.tile(array_all,2)
    if tc_data['lat_north'] > 1999:
        array_all = np.concatenate((array_all,np.zeros(array_all.shape)-31999), axis = 1)
    # array_all.shape (1, 2000, 5143)
    # 北侧的索引值更大
    array_tc_original = np.ascontiguousarray(array_all[:,tc_data['lat_north']:tc_data['lat_south']-1:-1 ,west:east])
    array_tc = array_tc_original * 0.01 + 200
    return array_tc

In [20]:
# for idx in match_info.index:    
#     tc_data = match_info.loc[idx]
#     nc_path = f"{root_path}{tc_data['YEAR']}/GRIDSAT-B1.{tc_data['time_dot']}.v02r01.nc"  
#     logger.info(nc_path)
#     if os.path.exists(nc_path):
#         # 读nc文件的irwin_2
#         ir = processing_nc(varible_name='irwin_cdr')
#         wvp = processing_nc(varible_name='irwvp')
#         vs = processing_nc(varible_name='vschn')
#         array_tc = np.concatenate([ir,wvp,vs],axis=0)
#         path_sid = f"{new_path}{tc_data['SID']}"
#         array_file = f"{path_sid}/{tc_data['SID']}.{tc_data['YEAR']}{tc_data['TIME']}00.npy"
#         os.makedirs(path_sid, exist_ok=True)
#         if os.path.exists(array_file) is False: 
#             np.save(array_file, array_tc)
#             logger.info(f"Numpy array saved,{array_file}")
#         else:
#             logger.info(f"Numpy array exist,{array_file}")
#     else:
#         logger.info(f"File Error!,{nc_path}")

In [21]:
min_img = 10
max_img = 10

ibtr_df = pd.DataFrame(columns = [
    "SID", "NAME", "BASIN", "LAT", "LON", "YEAR", "MONTH", "DAY", "HOUR", "PATH", "WIND_SPEED", "SIMPSON_CLASSIFICATION"
])

In [22]:
def SimpsonHurricane(wndsp):
    if wndsp <= 34:
        return 0
    elif 35 <= wndsp <=63:
        return 1
    elif 64 <= wndsp <= 82:
        return 2
    elif 83 <= wndsp <= 95:
        return 3
    elif 96<= wndsp <= 112:
        return 4
    elif 113<= wndsp <= 136:
        return 5
    else:
        return 6

# Buliding Dataset

In [23]:
# cnt = 0
# for i in tqdm(range(len(match_info))):
#     iso_time = match_info.at[i, "ISO_TIME"].strftime('%Y-%m-%d %H:%M:%S')
#     year = iso_time[:4]
#     month = iso_time[5:7]
#     day = iso_time[8:10]
#     hour = iso_time[11:13]
#     npy_name = match_info.at[i, "SID"] + "." + year + month + day + hour + "00.npy"
#     npy_path = new_path+match_info.at[i, "SID"] + '/' + npy_name
#     if os.path.exists(npy_path) == True:
#         ibtr_df.loc[cnt]={
#             "SID": match_info.at[i, "SID"],
#             "NAME":match_info.at[i, "NAME"],
#             "BASIN":match_info.at[i, "BASIN"],
#             "LAT":match_info.at[i, "LAT"],
#             "LON":match_info.at[i, "LON"],
#             "YEAR":year,
#             "MONTH":month,
#             "DAY":day,
#             "HOUR":hour+"00",
#             "PATH":npy_path,
#             "WIND_SPEED":float(match_info.at[i, "USA_WIND"]),
#             "SIMPSON_CLASSIFICATION":SimpsonHurricane(match_info.at[i, "USA_WIND"]),
#         }
#         cnt += 1       

In [24]:
ibtr_df = pd.DataFrame(columns = [
    "SID", "NAME", "BASIN", "LAT", "LON", "YEAR", "MONTH", "DAY", "HOUR", "PATH", "WIND_SPEED", "SIMPSON_CLASSIFICATION"
])
ibtr_df["SID"] = match_info["SID"]
ibtr_df["NAME"] = match_info["NAME"]
ibtr_df["BASIN"] = match_info["BASIN"]
ibtr_df["LAT"] = match_info["LAT"]
ibtr_df["LON"] = match_info["LON"]
ibtr_df["YEAR"] = match_info["ISO_TIME"].map(lambda x: int(x.strftime('%Y-%m-%d %H:%M:%S')[:4]))
ibtr_df["MONTH"] = match_info["ISO_TIME"].map(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')[5:7])
ibtr_df["DAY"] = match_info["ISO_TIME"].map(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')[8:10])
ibtr_df["HOUR"] = match_info["ISO_TIME"].map(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')[11:13]+"00")
ibtr_df["PATH"] = ibtr_df.apply(lambda x: new_path + x["SID"] + '/' + x["SID"] + "." + str(x["YEAR"]) + x["MONTH"] + x["DAY"] + x["HOUR"] + ".npy", axis=1) 
ibtr_df["WIND_SPEED"] = match_info["USA_WIND"]
ibtr_df["SIMPSON_CLASSIFICATION"] = match_info["USA_WIND"].apply(SimpsonHurricane)

temp = ibtr_df["PATH"].apply(os.path.exists)
temp[temp == False].shape,temp[temp == True].shape
ibtr_df.drop(labels=temp[temp == False].index, inplace=True)
ibtr_df.reset_index(drop=True, inplace=True)
ibtr_df

,SID,NAME,BASIN,LAT,LON,YEAR,MONTH,DAY,HOUR,PATH,WIND_SPEED,SIMPSON_CLASSIFICATION
0,2010002S09096,EDZANI,SI,-9.20000,95.7000,2010,01,01,1200,/home/ices/work/typhoon_intensity/data/GridSat...,15.0,0
1,2010002S09096,EDZANI,SI,-9.51501,95.5176,2010,01,01,1500,/home/ices/work/typhoon_intensity/data/GridSat...,20.0,0
2,2010002S09096,EDZANI,SI,-9.80000,95.3000,2010,01,01,1800,/home/ices/work/typhoon_intensity/data/GridSat...,25.0,0
3,2010002S09096,EDZANI,SI,-10.03000,95.0225,2010,01,01,2100,/home/ices/work/typhoon_intensity/data/GridSat...,25.0,0
4,2010002S09096,EDZANI,SI,-10.20000,94.7000,2010,01,02,0000,/home/ices/work/typhoon_intensity/data/GridSat...,25.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
57768,2020360S16057,CHALANE,SI,-19.60000,38.5000,2020,12,29,1200,/home/ices/work/typhoon_intensity/data/GridSat...,54.0,1
57769,2020360S16057,CHALANE,SI,-19.68650,38.1155,2020,12,29,1500,/home/ices/work/typhoon_intensity/data/GridSat...,57.0,1
57770,2020360S16057,CHALANE,SI,-19.70000,37.7000,2020,12,29,1800,/home/ices/work/typhoon_intensity/data/GridSat...,60.0,1
57771,2020360S16057,CHALANE,SI,-19.51400,36.9938,2020,12,29,2100,/home/ices/work/typhoon_intensity/data/GridSat...,60.0,1


# Drop Duplicate Records

In [25]:
ibtr_df.drop_duplicates(subset=['SID','YEAR','MONTH','DAY','HOUR'],keep='first',inplace=True)

# Drop Non-Series SID

In [26]:
sids = np.unique(ibtr_df['SID'])

In [27]:
len(sids)

1048

In [28]:
def num_2_str(num):
    return str(num).zfill(2)

In [29]:
def num_2_str4(num):
    return str(num).zfill(4)

In [30]:
invaild_sid = []
for sid in sids:
    typhoon = ibtr_df[ibtr_df['SID'] == sid]
    ty_datatime = typhoon['YEAR'].map(str) + typhoon['MONTH'].apply(num_2_str) + typhoon['DAY'].apply(num_2_str)+ typhoon['HOUR'].apply(num_2_str4)
    ty_datatime = ty_datatime.tolist()
    startitem = datetime.datetime.strptime(ty_datatime[0],"%Y%m%d%H%M")
    for i in range(1, len(ty_datatime)):
        nextitem = datetime.datetime.strptime(ty_datatime[i],"%Y%m%d%H%M")
        delta = nextitem - startitem
        startitem = nextitem
        if delta.seconds//3600 != 3:
            invaild_sid.append(sid)
            break

In [31]:
invaild_sid

['2018136N11054', '2019236N08143', '2020036S16066', '2020355N10115']

In [32]:
ibtr_df.shape

(57678, 12)

In [33]:
invalid_sid_index = ibtr_df[ibtr_df['SID'].isin(invaild_sid)].index
ibtr_df.drop(labels=invalid_sid_index, inplace=True)
ibtr_df.reset_index(inplace=True, drop=True)

In [34]:
ibtr_df.shape

(57504, 12)

# Invalid Value Impute

In [35]:
if os.path.exists(new2_path)==False:
    os.mkdir(new2_path)
for _dir in sorted(os.listdir(new_path)):
    if os.path.exists(new2_path+_dir) == False and _dir.find('.ipynb_checkpoints')==-1:
        os.mkdir(path=new2_path+_dir)

In [36]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5, weights="uniform")
invaild_img = []
for i in tqdm(range(len(ibtr_df))):
    npy_path = ibtr_df.at[i, "PATH"]
    
    item = np.load(npy_path)
    if len(np.where(item[0] < 0)[0])!=0:
        try:
            item[0] = imputer.fit_transform(np.where(item[0]<0,np.nan,item[0]))
        except:
            invaild_img.append(npy_path)
    npy_path_new = npy_path.replace('GridSat_B1_npy', 'GridSat_B1_new_npy')
    np.save(npy_path_new, item[0:0+1])

100%|██████████| 57504/57504 [10:56<00:00, 87.56it/s]  


In [37]:
invaild_img

['/home/ices/work/typhoon_intensity/data/GridSat_B1_npy/2011270N18139/2011270N18139.201109281500.npy',
 '/home/ices/work/typhoon_intensity/data/GridSat_B1_npy/2011302N13062/2011302N13062.201111020900.npy',
 '/home/ices/work/typhoon_intensity/data/GridSat_B1_npy/2011323N09264/2011323N09264.201111230000.npy',
 '/home/ices/work/typhoon_intensity/data/GridSat_B1_npy/2011361S09095/2011361S09095.201201010000.npy',
 '/home/ices/work/typhoon_intensity/data/GridSat_B1_npy/2012359N08134/2012359N08134.201212250300.npy',
 '/home/ices/work/typhoon_intensity/data/GridSat_B1_npy/2012359N08134/2012359N08134.201212250600.npy',
 '/home/ices/work/typhoon_intensity/data/GridSat_B1_npy/2012359N08134/2012359N08134.201212250900.npy',
 '/home/ices/work/typhoon_intensity/data/GridSat_B1_npy/2012359N08134/2012359N08134.201212251200.npy',
 '/home/ices/work/typhoon_intensity/data/GridSat_B1_npy/2012359N08134/2012359N08134.201212251500.npy',
 '/home/ices/work/typhoon_intensity/data/GridSat_B1_npy/2012359N08134/201

In [38]:
invaild_img_sids = []
for img in invaild_img:
    invaild_img_sids.append(img.split('/')[7])
invaild_img_sids = np.unique(invaild_img_sids)

for sid in invaild_img_sids:
    if os.path.exists(new2_path+sid):
        shutil.rmtree(new2_path+sid)

In [39]:
invaild_img_sids

array(['2011270N18139', '2011302N13062', '2011323N09264', '2011361S09095',
       '2012359N08134', '2014058N09149', '2014117N10145', '2014225N15255',
       '2014229N16246', '2015068S12151', '2015274N13201', '2015323S05080',
       '2016270N12223', '2016270N15240', '2016273N13300', '2017193N12252',
       '2017199N14255', '2017202N08274', '2017203N14248', '2018073S09129',
       '2018079S08137', '2018082N04147', '2018311N07179', '2018321N10113',
       '2019042S17162', '2019050N05163', '2020154N19269', '2020221N13262'],
      dtype='<U13')

In [40]:
ibtr_df.drop(ibtr_df[ibtr_df['SID'].isin(invaild_img_sids)].index, inplace=True)
ibtr_df['PATH'] = ibtr_df['PATH'].map(lambda x: x.replace('GridSat_B1_npy', 'GridSat_B1_new_npy'))
ibtr_df.reset_index(inplace=True, drop=True)
ibtr_df.shape

(55316, 12)

In [41]:
np.load(ibtr_df.loc[0, 'PATH']).shape

(1, 301, 301)

# Save

In [42]:
ibtr_df.to_csv('/home/ices/work/typhoon_intensity/data/gridsat.path.ibtr.windspeed.csv', index=False)

# Min Max

In [43]:
for i in tqdm(range(len(ibtr_df))):
    npy_path = ibtr_df.at[i, "PATH"]
    item = np.load(npy_path)
    if train_range[0] <= ibtr_df.at[i, "YEAR"] <= train_range[1]:
        item_min = np.min(item)
        item_max = np.max(item)
        if item_min < min_img:
            min_img = item_min
        if item_max > max_img:
            max_img = item_max

100%|██████████| 55316/55316 [01:44<00:00, 529.76it/s] 


In [44]:
min_img, max_img

(10, 348.31)

In [45]:
np.save('/home/ices/work/typhoon_intensity/data/gridsat.img.min.max.npy', np.array([min_img, max_img]))